In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime
# Import the API key.
from config import weather_api_key
import requests

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

642

In [7]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
    # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
    # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
    # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
    # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | puro
Processing Record 2 of Set 1 | yellowknife
Processing Record 3 of Set 1 | samusu
City not found. Skipping...
Processing Record 4 of Set 1 | broome
Processing Record 5 of Set 1 | chitral
Processing Record 6 of Set 1 | vaini
Processing Record 7 of Set 1 | brand-erbisdorf
Processing Record 8 of Set 1 | rikitea
Processing Record 9 of Set 1 | bluff
Processing Record 10 of Set 1 | lagoa
Processing Record 11 of Set 1 | ballina
Processing Record 12 of Set 1 | san quintin
Processing Record 13 of Set 1 | castro
Processing Record 14 of Set 1 | saint-philippe
Processing Record 15 of Set 1 | north platte
Processing Record 16 of Set 1 | kapaa
Processing Record 17 of Set 1 | zeya
Processing Record 18 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 19 of Set 1 | hilo
Processing Record 20 of Set 1 | punta arenas
Processing Record 21 of Set 1 | upernavik
Processing Record 22 of Set 1 | h

In [11]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puro,12.4841,123.3827,82.63,77,77,3.69,PH,2022-07-23 13:21:44
1,Yellowknife,62.4560,-114.3525,63.12,72,75,13.80,CA,2022-07-23 13:21:44
2,Broome,42.2506,-75.8330,78.48,74,0,8.05,US,2022-07-23 13:21:45
3,Chitral,35.8462,71.7858,83.01,62,97,7.47,PK,2022-07-23 13:21:45
4,Vaini,-21.2000,-175.2000,69.96,83,20,9.22,TO,2022-07-23 13:21:45
5,Brand-Erbisdorf,50.8667,13.3333,72.50,65,80,10.38,DE,2022-07-23 13:21:46
6,Rikitea,-23.1203,-134.9692,71.44,74,5,10.56,PF,2022-07-23 13:21:46
7,Bluff,-46.6000,168.3333,35.80,79,31,6.87,NZ,2022-07-23 13:21:46
8,Lagoa,39.0500,-27.9833,72.25,87,46,7.56,PT,2022-07-23 13:21:47
9,Ballina,-28.8667,153.5667,57.02,99,19,13.96,AU,2022-07-23 13:21:47


In [13]:
new_column_order = ["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]
ordered_city_data_df = city_data_df[new_column_order]
ordered_city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Puro,PH,2022-07-23 13:21:44,12.4841,123.3827,82.63,77,77,3.69
1,Yellowknife,CA,2022-07-23 13:21:44,62.4560,-114.3525,63.12,72,75,13.80
2,Broome,US,2022-07-23 13:21:45,42.2506,-75.8330,78.48,74,0,8.05
3,Chitral,PK,2022-07-23 13:21:45,35.8462,71.7858,83.01,62,97,7.47
4,Vaini,TO,2022-07-23 13:21:45,-21.2000,-175.2000,69.96,83,20,9.22


In [14]:
#create the csv file
output_data_file = "weather_data/cities.csv"
#export the data into the csv
ordered_city_data_df.to_csv(output_data_file, index_label ="City_ID")